In [1]:
import os

from ludwig.api import LudwigModel

import pandas as pd
import numpy as np

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Подготовка данных

In [4]:
! wget https://github.com/DeepLenin/fashion-mnist_png/raw/master/data.zip

--2020-06-14 22:25:54--  https://github.com/DeepLenin/fashion-mnist_png/raw/master/data.zip
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DeepLenin/fashion-mnist_png/master/data.zip [following]
--2020-06-14 22:25:55--  https://raw.githubusercontent.com/DeepLenin/fashion-mnist_png/master/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48027867 (46M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  45,80M  9,96MB/s    in 5,5s    

2020-06-14 22:26:04 (8,38 MB/s) - ‘data.zip’ saved [48027867/48027867]



In [5]:
! unzip data.zip

In [11]:
for name in ['train', 'test']:
    with open('mnist_dataset_{}.csv'.format(name), 'w') as output_file:
        print('=== creating {} dataset ==='.format(name))
        output_file.write('image_path,label\n')
        for i in range(10):
            path = os.path.abspath('./data/{}/{}'.format(name, i))
            for file in os.listdir(path):
                if file.endswith(".png"):
                    output_file.write('{},{}\n'.
                                      format(os.path.join(path, file), str(i)))

In [12]:
train_df = pd.read_csv('./mnist_dataset_train.csv')
test_df = pd.read_csv('./mnist_dataset_test.csv')

# Обучение модели 

In [26]:
model_definition = {'input_features':
                    [{'name': 'image_path', 'type': 'image',
                      'encoder': 'stacked_cnn',
                      'conv_layers': [{'num_filters': 32, 'filter_size': 3,
                                       'pool_size': 2, 'pool_stride': 2},
                                      {'num_filters': 64, 'filter_size': 3,
                                       'pool_size': 2, 'pool_stride': 2,
                                       'dropout': True}],
                     'fc_layers': [{'fc_size': 128,
                                   'dropout': True}]}],
                    'output_features':
                    [{'name': 'label', 'type': 'category'}],
                    'training': {'dropout_rate': 0.4}}

In [44]:
model = LudwigModel(model_definition)

In [46]:
train_stats = model.train(train_df)

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.



# Оценка на тестовых данных

In [65]:
test_results_df, test_statistics = model.test(test_df)

In [68]:
test_statistics.get('label').get('accuracy')

0.877